In [ ]:
import pandas as pd
from siphon.simplewebservice.igra2 import IGRAUpperAir
from scipy.interpolate import interp1d
import numpy as np

# Configuration dictionary
CONFIG = {
    'stations_file': 'data/european_stations_2010_2025.csv',
    'input_file': '../data/tile_40N_0E_synthetic_points_with_RHi.csv',
    'output_file': 'tile_40N_0E_synthetic_points_with_RHi_updated.csv'
}


def load_stations(config):
    df = pd.read_csv(config['stations_file'])
    df['start_datetime'] = pd.to_datetime(
        df['date_range'].str.split(' to ').str[0])
    df['end_datetime'] = pd.to_datetime(
        df['date_range'].str.split(' to ').str[1])
    return df


def load_synthetic_points(config):
    return pd.read_csv(config['input_file'], parse_dates=['timestamp'])


def fetch_radiosonde_data(station_id, timestamp):
    try:
        df, _ = IGRAUpperAir.request_data(timestamp, station_id)
        return df
    except Exception as e:
        print(f"Failed to fetch data for {station_id} at {timestamp}: {e}")
        return None


def interpolate_rhi(sounding_data, target_altitude_m):
    if sounding_data is None or sounding_data.empty:
        return np.nan
    heights = sounding_data['height'].dropna().values
    rh = sounding_data['relative_humidity'].dropna().values
    if len(heights) < 2 or len(rh) < 2:
        return np.nan
    interpolator = interp1d(
        heights, rh, bounds_error=False, fill_value=(rh[0], rh[-1]))
    return float(interpolator(target_altitude_m))


def process_synthetic_points(config):
    print("Loading stations data...")
    stations_df = load_stations(config)
    print(f"Loaded {len(stations_df)} stations.")

    print("Loading synthetic points data...")
    df = load_synthetic_points(config)
    print(f"Loaded {len(df)} data points.")

    # Add new column for station altitude
    if 'altitude_m_station' not in df.columns:
        df['altitude_m_station'] = np.nan

    # Match station elevation to synthetic points
    for index, row in df.iterrows():
        station_id = row['nearest_station_id']
        timestamp = row['timestamp']
        station_info = stations_df[
            (stations_df['station_id'] == station_id) &
            (stations_df['start_datetime'] == timestamp)
        ]
        if not station_info.empty:
            elevation_m = station_info['elevation'].iloc[0]
            df.at[index, 'altitude_m_station'] = elevation_m
        else:
            print(f"No matching station data for {station_id} at {timestamp}")

    # Ensure RHi_true column exists
    if 'RHi_true' not in df.columns:
        df['RHi_true'] = np.nan

    # Process each row for RHi interpolation using station altitude
    for index, row in df.iterrows():
        print(f"Processing row {index + 1}/{len(df)}...")
        station_id = row['nearest_station_id']
        timestamp = row['timestamp']
        altitude_m = row['altitude_m_station']
        if pd.isna(altitude_m):
            print(
                f"Skipping row {index + 1}: No elevation found for {station_id} at {timestamp}")
            continue
        sounding_data = fetch_radiosonde_data(station_id, timestamp)
        rhi_value = interpolate_rhi(sounding_data, altitude_m)
        df.at[index, 'RHi_true'] = rhi_value
        print(f"Updated RHi_true for {station_id} at {timestamp}: {rhi_value}")

    # Save the updated DataFrame
    print("Saving updated data...")
    df.to_csv(config['output_file'], index=False)
    print(f"Data saved to {config['output_file']}")


if __name__ == "__main__":
    process_synthetic_points(CONFIG)

Loading stations data...
Loaded 9 stations.
Loading synthetic points data...
Loaded 200 data points.
Processing row 1/200...
Updated RHi_true for GMM00010113 at 2011-06-20 18:00:00: nan
Processing row 2/200...
Updated RHi_true for ITM00016546 at 2012-02-28 12:00:00: nan
Processing row 3/200...
Updated RHi_true for ITM00016546 at 2012-02-28 12:00:00: nan
Processing row 4/200...
Updated RHi_true for ITM00016037 at 2010-01-04 00:00:00: nan
Processing row 5/200...
Updated RHi_true for GMM00010113 at 2011-06-20 18:00:00: nan
Processing row 6/200...
Updated RHi_true for GMM00010113 at 2011-06-20 18:00:00: nan
Processing row 7/200...
Updated RHi_true for ITM00016037 at 2010-01-04 00:00:00: nan
Processing row 8/200...
Updated RHi_true for ITM00016546 at 2012-02-28 12:00:00: nan
Processing row 9/200...
Updated RHi_true for SPM00008383 at 2018-12-12 00:00:00: nan
Processing row 10/200...
Updated RHi_true for SPM00008383 at 2018-12-12 00:00:00: nan
Processing row 11/200...
Updated RHi_true for GM

In [13]:
import os
import pandas as pd
import numpy as np
import xarray as xr
from tqdm import tqdm

# INPUTS
CSV_INPUT_PATH = "../data/tile_40N_0E_synthetic_points_with_RHi.csv"
CSV_OUTPUT_PATH = "../data/8x113.csv"
MERRA2_FOLDER = "../data/merra2_files"


def get_unique_dates(df):
    timestamps = pd.to_datetime(df['timestamp'])
    return sorted(set(timestamps.dt.date))


def load_merra2_dataset(date):
    filename = f"MERRA2_{date}.nc4"
    filepath = os.path.join(MERRA2_FOLDER, filename)
    if not os.path.exists(filepath):
        print(f"⚠️ Missing file {filepath}")
        return None
    try:
        ds = xr.open_dataset(filepath)
        return ds
    except Exception as e:
        print(f"❌ Could not open {filepath}: {e}")
        return None


def extract_rh(ds, lat, lon, timestamp):
    try:
        # MERRA-2 convention: latitude from -90 to 90, longitude from 0 to 360
        if lon < 0:
            lon += 360

        # Find nearest time index
        time_var = ds['time']
        # Convert timestamp to pandas Timestamp
        point_time = pd.to_datetime(timestamp)
        # Find nearest time in the dataset
        time_diffs = np.abs(pd.to_datetime(time_var.values) - point_time)
        time_idx = time_diffs.argmin()

        # Find nearest lat/lon indices
        lat_idx = np.abs(ds['lat'].values - lat).argmin()
        lon_idx = np.abs(ds['lon'].values - lon).argmin()

        # Extract RHI value
        rh = ds['RHI'].isel(time=time_idx, lat=lat_idx,
                            lon=lon_idx).values.item()
        return float(rh)
    except Exception as e:
        print(f"⚠️ Error extracting RH: {e}")
        return np.nan


def main():
    # 1. Load CSV
    print(f"📂 Loading {CSV_INPUT_PATH}")
    df = pd.read_csv(CSV_INPUT_PATH)
    print(f"✅ Loaded {len(df)} rows.")

    # 2. Find unique dates
    unique_dates = get_unique_dates(df)
    print(f"📅 Found {len(unique_dates)} unique dates.")

    # 3. Load all MERRA2 files
    print(f"📥 Loading MERRA-2 files...")
    merra2_data = {}
    for date in tqdm(unique_dates, desc="Loading files"):
        ds = load_merra2_dataset(date)
        if ds:
            merra2_data[date] = ds

    # 4. Fill RHi_true
    df['RHi_true'] = np.nan
    print(f"🧹 Filling RHi_true values...")
    for idx in tqdm(df.index, desc="Filling RHi_true"):
        row = df.loc[idx]
        date = pd.to_datetime(row['timestamp']).date()
        ds = merra2_data.get(date)
        if ds:
            rhi_value = extract_rh(
                ds, row['lat'], row['lon'], row['timestamp'])
            df.at[idx, 'RHi_true'] = rhi_value

    # 5. Save
    df.to_csv(CSV_OUTPUT_PATH, index=False)
    print(f"✅ Saved final CSV to {CSV_OUTPUT_PATH}")

    # 6. Close datasets
    for ds in merra2_data.values():
        ds.close()


if __name__ == "__main__":
    main()

📂 Loading ../data/tile_40N_0E_synthetic_points_with_RHi.csv
✅ Loaded 200 rows.
📅 Found 4 unique dates.
📥 Loading MERRA-2 files...


Loading files: 100%|██████████| 4/4 [00:00<00:00, 160.50it/s]


❌ Could not open ../data/merra2_files\MERRA2_2010-01-04.nc4: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html
❌ Could not open ../data/merra2_files\MERRA2_2011-06-20.nc4: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html
❌ Could not open ../data/merra2_files\MERRA2_2012-02-28.nc4: did not find a match in any of xarray's cur

Filling RHi_true:   0%|          | 0/200 [00:00<?, ?it/s]

Filling RHi_true: 100%|██████████| 200/200 [00:00<00:00, 803.83it/s]

✅ Saved final CSV to ../data/8x113.csv


In [15]:
import pandas as pd
import cdsapi
import os
import xarray as xr
from tqdm import tqdm

# --- Configuration ---
CSV_INPUT_PATH = "../data/tile_40N_0E_synthetic_points_with_RHi.csv"
CSV_OUTPUT_PATH = "../data/8x113.csv"
ERA5_TEMP_FOLDER = "./era5_temp_gribs"
PRESSURE_LEVEL = 500  # in hPa
DOWNLOAD_TIME = "12:00"  # Closest to midday
VARIABLE = "relative_humidity"

# --- Setup ---
os.makedirs(ERA5_TEMP_FOLDER, exist_ok=True)

# --- Helper Functions ---


def fetch_era5_rhi(lat, lon, date):
    """Fetch RHi at given point, date and pressure level using ERA5 via cdsapi."""
    c = cdsapi.Client()

    year, month, day = date[:4], date[5:7], date[8:10]
    out_file = os.path.join(
        ERA5_TEMP_FOLDER, f"rhi_{lat:.2f}_{lon:.2f}_{year}{month}{day}.grib")

    if not os.path.exists(out_file):
        c.retrieve(
            'reanalysis-era5-pressure-levels',
            {
                'product_type': 'reanalysis',
                'variable': VARIABLE,
                'pressure_level': str(PRESSURE_LEVEL),
                'year': year,
                'month': month,
                'day': day,
                'time': DOWNLOAD_TIME,
                'format': 'grib',
                'area': [lat, lon, lat, lon],  # N,W,S,E
            },
            out_file
        )
    return out_file


def extract_rhi_from_grib(grib_file):
    """Extract RHi from a GRIB file."""
    ds = xr.open_dataset(grib_file, engine='cfgrib')
    rhi_value = ds['r'].values.item()
    return float(rhi_value)


# --- Load CSV ---
df = pd.read_csv(CSV_INPUT_PATH)

# --- Process Each Point ---
filled_rhis = []

print("🧹 Fetching RHi values and filling...")

for idx, row in tqdm(df.iterrows(), total=len(df)):
    lat = row['lat']
    lon = row['lon']
    timestamp = row['timestamp']

    try:
        grib_path = fetch_era5_rhi(lat, lon, timestamp)
        rhi = extract_rhi_from_grib(grib_path)
        filled_rhis.append(rhi)
    except Exception as e:
        print(f"⚠️ Failed at index {idx}: {e}")
        filled_rhis.append(None)

# --- Update CSV ---
df['RHi_true'] = filled_rhis

# --- Save Final CSV ---
df.to_csv(CSV_OUTPUT_PATH, index=False)

print(f"✅ Done. CSV saved to {CSV_OUTPUT_PATH}")

🧹 Fetching RHi values and filling...


  0%|          | 1/200 [00:06<21:11,  6.39s/it]

⚠️ Failed at index 0: Not Found


2025-04-29 21:40:02,441 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
  1%|          | 2/200 [00:06<09:31,  2.89s/it]2025-04-29 21:40:02,868 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels


⚠️ Failed at index 1: Not Found


...

⚠️ Failed at index 197: Not Found


2025-04-29 21:41:28,252 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
100%|█████████▉| 199/200 [01:32<00:00,  2.39it/s]

⚠️ Failed at index 198: Not Found


2025-04-29 21:41:28,669 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
100%|██████████| 200/200 [01:33<00:00,  2.15it/s]

⚠️ Failed at index 199: Not Found
✅ Done. CSV saved to ../data/8x113.csv


In [17]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': 'temperature',
        'pressure_level': '1000',
        'year': '2024',
        'month': '03',
        'day': '01',
        'time': '12:00',
    },
    'download.grib'
)

2025-04-29 21:59:47,974 INFO Sending request to https://cds.climate.copernicus.eu/api/resources/reanalysis-era5-pressure-levels


Exception: Not Found